# Decoradores da bibilioteca padrão: `lru_cache`

In [12]:
from functools import lru_cache
from time import perf_counter

@lru_cache
def fibonacci(n: int) -> int:
    if n < 2: return n
    return fibonacci(n-1) + fibonacci(n-2)

In [8]:
N = 35

## Fibonacci recursivo do `N`-ésimo item (padrão)

In [9]:
_t0 = perf_counter()
for i in range(N):
    fibonacci(i)
dt = 1000 * (perf_counter() - _t0)
print('\n\tTempo: %.5f ms' %dt)


	Tempo: 1839.90212 ms


## Fibonacci recursivo do `N`-ésimo item com cache

O cache foi implementado usando o decorator.

In [13]:
_t0 = perf_counter()
for i in range(N):
    fibonacci(i)
dt2 = 1000*(perf_counter() - _t0)
print('\n\tTempo: %.5f ms' %dt2)


	Tempo: 0.08179 ms


In [14]:
dt / dt2

22494.891018778733

**Resultado**: com o cache, temos um código 22 mil vezes mais rápido.

# Decoradores da bibilioteca padrão: `singledispatch`

Temos a função `htmlize`

In [1]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return f"<pre>{content}</pre>"

htmlize([2,4,6])

'<pre>[2, 4, 6]</pre>'

mas queremos, agora, o seguinte comportamento:

`>>> htmlize([2, 'ok', True])`

```html
<ul>
  <li>2</li>
  <li><pre>ok</pre></li>
  <li><pre>True</pre></li>
</ul>
```

só que, em python, não há sobrecarga de funções.

In [2]:
from functools import singledispatch

O decorator `singledispatch` implementa uma forma de resolver isto.

Primeiro, decoramos a função geral.

In [3]:
@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return f"<pre>{content}</pre>"

O processo de especialização ocorre como os dois exemplos abaixo (um pra texto e outro para número):

In [6]:
@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return f"<p>{content}</p>"

In [11]:
@htmlize.register(int)
@htmlize.register(float) # sim, podemos empilhar decoradores
def _(n):
    return f"<pre class='number'>{n}</pre>"

In [12]:
htmlize(19)

"<pre class='number'>19</pre>"

In [13]:
htmlize('Olá mundo!')

'<p>Olá mundo!</p>'

In [14]:
htmlize([4,3,5])

'<pre>[4, 3, 5]</pre>'

In [18]:
@htmlize.register(tuple)
@htmlize.register(list)
@htmlize.register(set)
def _(lista):
    content = "\n".join(f'<li>{htmlize(item)}</li>' for item in lista)
    return f"<ul>\n{content}\n</ul>"

In [19]:
print(htmlize([4,3,5]))

<ul>
<li><pre class='number'>4</pre></li>
<li><pre class='number'>3</pre></li>
<li><pre class='number'>5</pre></li>
</ul>


Legal, né?